In [138]:
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model.logistic import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier

import pickle
import tensorflow as tf
import numpy as np

from keras.layers import Lambda,Reshape,concatenate,Input, Embedding, LSTM
from keras.layers import Dense,Dropout, Activation ,Flatten ,RepeatVector, Bidirectional,GlobalAveragePooling1D
from keras.models import Model
from keras import backend as K
from keras.optimizers import Adam,SGD
from keras.activations import softmax
from keras import regularizers

from keras import backend as K, regularizers, constraints, initializers
from keras.engine.topology import Layer

from keras.layers import merge
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

from keras.callbacks import EarlyStopping


from Attention import Attention

from keras.layers import Concatenate,Dot
from keras.layers import Permute, merge

from string import punctuation

import re

from nltk.corpus import stopwords
from nltk import word_tokenize

from nltk import pos_tag
from string import punctuation,digits

In [139]:
def remove_punctuation(s):
    list_punctuation = list(punctuation)
    for i in list_punctuation:
        s = s.replace(i,'')
    return s.lower()


def clean_sentence(sentence):
    sentence = sentence.lower()
    sentence = re.sub(r'(\W)\1{2,}', r'\1', sentence) 
    sentence = re.sub(r'(\w)\1{2,}', r'\1\1', sentence)
    sentence = re.sub(r'(?P<url>https?://[^\s]+)', 'URL', sentence)
    sentence = re.sub(r"\@(\w+)", 'MENTION', sentence)
    sentence = sentence.replace('#(\w+)','HASHTAG')
    sentence = sentence.replace('$(\w+)','TARGET')
    sentence = sentence.replace("'s",' ')
    sentence = sentence.replace("-",' ')
    tokens = sentence.split()
    tokens = [remove_punctuation(w) for w in tokens]
    stop_words = set(stopwords.words('english'))
    tokens = [w for w in tokens if not w in stop_words]
    remove_digits = str.maketrans('', '', digits)
    tokens = [w.translate(remove_digits) for w in tokens]
    tokens = [w.strip() for w in tokens]
    tokens = [w for w in tokens if w!=""]
    tokens = ' '.join(tokens)
    return tokens

In [140]:
dataset1=pickle.load(open(r"D:\PythonCodes\Sentiment-Analysis\Data\train_data_augmented.dat","rb"))
head=pickle.load(open(r"D:\PythonCodes\Sentiment-Analysis\Data\head.dat","rb"))
post=pickle.load(open(r"D:\PythonCodes\Sentiment-Analysis\Data\post.dat","rb"))

a=pickle.load(open(r"D:\PythonCodes\Sentiment-Analysis\Data\all_data.dat","rb"))

In [141]:
corpus=dataset1['sentence']

In [142]:
a.keys()

dict_keys(['p_sentiment', 'aug_aspect', 'lable_encoding', 'h_aspect_encoding', 'text_raw', 'h_target', 'train_sentiment', 'h_aspect', 'input_aspect', 'validation_X', 'p_aspect', 'h_sentence', 'embedding_matrix', 'aspect_level1', 'validation_Y', 'p_target', 'vocab_size', 'aug_sentence', 'aspect_level2', 'p_sentence', 'aspect', 'test_raw', 'sentence', 'target', 'h_sentiment', 'p_aspect_encoding'])

In [143]:
def get_class_from_pred(pred):
    return [label_encoding[x.argmax()] for x in pred]

In [144]:
input_context=a['aug_sentence']
#input_target=a['target']

label_encoding=a['lable_encoding']

embedding_matrix=a['embedding_matrix']

sentiment= a['train_sentiment']
aspect_encoded=a['aug_aspect']

h_sentence=a['h_sentence']
h_aspect_encoding=a['h_aspect_encoding']
h_sentiment= a['h_sentiment']


p_sentence=a['p_sentence']
p_aspect_encoding=a['p_aspect_encoding']
p_sentiment= a['p_sentiment']



y=[]
for i in h_aspect_encoding:
    y.append(list(i))
for j in p_aspect_encoding:
    y.append(list(j)) 
print(len(y))
y=np.array(y)

validation_X=a['validation_X']
vocab_size=a['vocab_size']



192


In [145]:
validation_text= []
for i in head['h_sentence']:
    validation_text.append(clean_sentence(i))


In [146]:
len(embedding_matrix)

3727

In [147]:
aspect=dataset1['aspect']
input_text=[]
for i in dataset1['sentence']:
    input_text.append(clean_sentence(i))

In [148]:
input_text[0]

'amzn going thru roof needs sell'

In [149]:
vectorizer = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='latin-1', ngram_range=(1, 2), stop_words='english')
X_train = vectorizer.fit_transform(input_text)
classifier = LogisticRegression()
classifier.fit(X_train,aspect)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [150]:
X_test = vectorizer.transform(head['h_sentence'])
predictions = classifier.predict(X_test)
f1_score(head['aspect'],predictions,average='macro')

C:\Users\simcy\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


0.1470458553791887

In [151]:

forest = RandomForestClassifier(n_estimators = 300) 
forest = forest.fit( X_train, aspect)
predictions = forest.predict(X_test)
f1_score(head['aspect'],predictions,average='macro')

C:\Users\simcy\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\simcy\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


0.10545106931661555

In [47]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB

count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(input_text)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

clf = MultinomialNB().fit(X_train_tfidf, aspect)

In [51]:
predictions=clf.predict(count_vect.transform(head['h_sentence']))
f1_score(head['aspect'],predictions,average='macro')

C:\Users\simcy\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\simcy\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


0.10949252615038314

In [52]:
from sklearn.svm import LinearSVC

model = LinearSVC()
model.fit(X_train, aspect)
y_pred = model.predict(X_test)

In [53]:
f1_score(head['aspect'],y_pred,average='macro')

C:\Users\simcy\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\simcy\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


0.12576378963938772

# Adding Attention to our model

In [82]:
def attention_2(learning_rate,dropout,lstm_out,n_hidden_layer,em,em_trainable_flag,em_dim):
     
    input_context= Input(shape=(max_length,),name='Context')
    
    embedding=Embedding(vocab_size, len(eval(em)[0]), weights = [eval(em)],input_length=max_length,trainable = em_trainable_flag)
    context= embedding(input_context)
        
    a = Bidirectional(LSTM(300, return_sequences=True,recurrent_dropout=dropout))(context)
    
    print(a.shape)

    alpha = Attention()(a)
    
    x=Dense(int((2*lstm_out+27)/2),activation='relu')(alpha)
        
    out=Dense(27,activation='softmax')(x)
    
    model= Model(inputs=input_context ,outputs=out)
    
    optimizer = Adam(lr=learning_rate)
    
    model.compile(loss = 'categorical_crossentropy', optimizer=optimizer,metrics = ['accuracy'])
    
    
    print(model.summary())
    
    return model

In [83]:

attention_2 = attention_2(learning_rate=0.00069,
                     dropout=0.5,
                     lstm_out=300,
                     n_hidden_layer=1,
                     em='embedding_matrix',
                     em_trainable_flag=False,
                     em_dim=300)

(?, ?, 600)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Context (InputLayer)         (None, 11)                0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 11, 300)           1118100   
_________________________________________________________________
bidirectional_2 (Bidirection (None, 11, 600)           1442400   
_________________________________________________________________
attention_2 (Attention)      (None, 600)               611       
_________________________________________________________________
dense_3 (Dense)              (None, 313)               188113    
_________________________________________________________________
dense_4 (Dense)              (None, 27)                8478      
Total params: 2,757,702
Trainable params: 1,639,602
Non-trainable params: 1,118,100
______________________________________________

In [84]:

attention_2.fit(x=a['sentence'],y=aspect, epochs=100,batch_size=16,callbacks=[earlystopper])
    

Epoch 1/100
1343/1343 [==============================] - 16s 12ms/step - loss: 2.5115 - acc: 0.3552
Epoch 2/100
1343/1343 [==============================] - 12s 9ms/step - loss: 1.6936 - acc: 0.5443
Epoch 3/100
1343/1343 [==============================] - 13s 10ms/step - loss: 1.2348 - acc: 0.6575
Epoch 4/100
1343/1343 [==============================] - 13s 9ms/step - loss: 0.9288 - acc: 0.7215
Epoch 5/100
1343/1343 [==============================] - 13s 10ms/step - loss: 0.7106 - acc: 0.7870
Epoch 6/100
1343/1343 [==============================] - 13s 9ms/step - loss: 0.5657 - acc: 0.8310
Epoch 7/100
1343/1343 [==============================] - 13s 10ms/step - loss: 0.4455 - acc: 0.8675
Epoch 8/100
1343/1343 [==============================] - 13s 10ms/step - loss: 0.3710 - acc: 0.8928
Epoch 9/100
1343/1343 [==============================] - 13s 9ms/step - loss: 0.2964 - acc: 0.9077
Epoch 10/100
1343/1343 [==============================] - 13s 9ms/step - loss: 0.2560 - acc: 0.9248
Epoc

In [86]:
predictions=attention_2.predict(h_sentence)
predictions=get_class_from_pred(predictions)
f1_score(head['aspect'],predictions,average='macro')

C:\Users\simcy\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\simcy\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


0.1208631889552942

In [158]:
length=11
def cnn_4(dropout,learning_rate,em,em_dim,em_trainable_flag,num_filters=150):
    
        # channel 1
        inputs1 = Input(shape=(length,))
        embedding1 = Embedding(vocab_size, 300)(inputs1)
        
        q = Bidirectional(LSTM(327,return_sequences=True,recurrent_dropout=dropout))(embedding1)
        q = Bidirectional(LSTM(327,return_sequences=True,recurrent_dropout=dropout))(q)



        conv1 = Conv1D(filters=num_filters, kernel_size=1, activation='relu')(q)
        drop1 = Dropout(0.2)(conv1)
        pool1 = MaxPooling1D(pool_size=2)(drop1)
        flat1 = Flatten()(pool1)
        #q1 = Bidirectional(LSTM(300,return_sequences=True,recurrent_dropout=dropout))(pool1)

        
        # channel 2
        inputs2 = Input(shape=(length,))
        embedding2 = Embedding(vocab_size, 300)(inputs2)
        conv2 = Conv1D(filters=num_filters, kernel_size=2, activation='relu')(embedding2)
        drop2 = Dropout(0.2)(conv2)
        pool2 = MaxPooling1D(pool_size=2)(drop2)
        flat2 = Flatten()(pool2)
        #q2 = Bidirectional(LSTM(300,return_sequences=True,recurrent_dropout=dropout))(pool2)

        # channel 3
        inputs3 = Input(shape=(length,))
        embedding3 = Embedding(vocab_size, 300)(inputs3)
        conv3 = Conv1D(filters=num_filters, kernel_size=3, activation='relu')(embedding3)
        drop3 = Dropout(0.2)(conv3)
        pool3 = MaxPooling1D(pool_size=2)(drop3)
        flat3 = Flatten()(pool3)
        #q3 = Bidirectional(LSTM(300,return_sequences=False,recurrent_dropout=dropout))(pool3)

        
        # merge
        merged = concatenate([flat1, flat2, flat3])
        # interpretation
        
        dense1 = Dense(80, activation='relu')(merged)
        outputs = Dense(27, activation='softmax')(dense1)
        model = Model(inputs=[inputs1,inputs2,inputs3], outputs=outputs)
        # compile
        optimizer=Adam(lr=learning_rate)
        model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
        # summarize
        print(model.summary())
        return model

In [159]:
cnn_model_4 = cnn_4(dropout=0.2,
                     learning_rate=0.001,
                     em='embedding_matrix',
                     em_trainable_flag=False,
                     em_dim=300)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_32 (InputLayer)           (None, 11)           0                                            
__________________________________________________________________________________________________
embedding_34 (Embedding)        (None, 11, 300)      1118100     input_32[0][0]                   
__________________________________________________________________________________________________
bidirectional_16 (Bidirectional (None, 11, 654)      1642848     embedding_34[0][0]               
__________________________________________________________________________________________________
input_33 (InputLayer)           (None, 11)           0                                            
__________________________________________________________________________________________________
input_34 (

In [162]:
earlystopper = EarlyStopping(monitor='loss', patience=5, verbose=1)


In [163]:

cnn_model_4.fit(x=[a['sentence'],a['sentence'],a['sentence']],y=aspect_encoded, epochs=50,batch_size=8,callbacks=[earlystopper],validation_data=([h_sentence,h_sentence,h_sentence],h_aspect_encoding))
    

Train on 1343 samples, validate on 93 samples
Epoch 1/50
1343/1343 [==============================] - 76s 57ms/step - loss: 0.1249 - acc: 0.9661 - val_loss: 0.1740 - val_acc: 0.9602
Epoch 2/50
1343/1343 [==============================] - 68s 51ms/step - loss: 0.0780 - acc: 0.9773 - val_loss: 0.1797 - val_acc: 0.9586
Epoch 3/50
1343/1343 [==============================] - 68s 51ms/step - loss: 0.0298 - acc: 0.9909 - val_loss: 0.2255 - val_acc: 0.9550
Epoch 4/50
1343/1343 [==============================] - 69s 51ms/step - loss: 0.0073 - acc: 0.9979 - val_loss: 0.2347 - val_acc: 0.9546
Epoch 5/50
1343/1343 [==============================] - 69s 51ms/step - loss: 0.0029 - acc: 0.9994 - val_loss: 0.2525 - val_acc: 0.9518
Epoch 6/50
1343/1343 [==============================] - 68s 51ms/step - loss: 0.0024 - acc: 0.9996 - val_loss: 0.2641 - val_acc: 0.9522
Epoch 7/50
1343/1343 [==============================] - 68s 51ms/step - loss: 0.0016 - acc: 0.9996 - val_loss: 0.2664 - val_acc: 0.9490
Ep

KeyboardInterrupt: 

In [164]:
predictions=cnn_model_4.predict([h_sentence,h_sentence,h_sentence])
predictions=get_class_from_pred(predictions)
f1_score(head['aspect'],predictions,average='macro')

C:\Users\simcy\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\simcy\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


0.0738205365402405

In [165]:
predictions=cnn_model_4.predict([p_sentence,p_sentence,p_sentence])
predictions=get_class_from_pred(predictions)
f1_score(post['aspect'],predictions,average='macro')

C:\Users\simcy\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


0.1533531746031746

n_filters = 100
    h_sentence :0.106
    p_sentence :0.204
n_filters = 150 
    h_sentence :0.132
    p_sentence :0.195
n_dense_layer = 317
    h_sentence = 0.091
    p=0.29
1 Bi LSTM LAYER 
h -0.111
p-0.153

# Character level CNN

In [74]:
import numpy as np
import re
import csv


class Data(object):
    """
    Class to handle loading and processing of raw datasets.
    """
    def __init__(self, data_source,aspect_source,
                 alphabet="abcdefghijklmnopqrstuvwxyz0123456789-,;.!?:'\"/\\|_@#$%^&*~`+-=<>()[]{}",
                 input_size=1014, num_of_classes=27):
        """
        Initialization of a Data object.
        Args:
            alphabet (str): Alphabet of characters to index
            input_size (int): Size of input features
            num_of_classes (int): Number of classes in data
        """
        self.alphabet = alphabet
        self.alphabet_size = len(self.alphabet)
        self.dict = {}  # Maps each character to an integer
        self.no_of_classes = num_of_classes
        for idx, char in enumerate(self.alphabet):
            self.dict[char] = idx + 1
        self.length = input_size
        self.data_source=data_source
        self.aspect_source=aspect_source

    def load_data(self):
        """
        Load raw data from the source file into data variable.
        Returns: None
        """
        data = []
        
        for row in self.data_source:
                txt = ""
                for s in row:
                    txt = txt + " " + re.sub("^\s*(.-)\s*$", "%1", s).replace("\\n", "\n")
                data.append(txt)  # format: (label, text)
                
        self.data = np.array(data)
        print("Data loaded ")

    def get_all_data(self):
        """
        Return all loaded data from data variable.
        Returns:
            (np.ndarray) Data transformed from raw to indexed form with associated one-hot label.
        """
        data_size = len(self.data)
        start_index = 0
        end_index = data_size
        batch_texts = self.data[start_index:end_index]
        batch_indices = []
        one_hot = np.eye(self.no_of_classes, dtype='int64')
        classes = []
        for s in batch_texts:
            batch_indices.append(self.str_to_indexes(s))
            
        return np.asarray(batch_indices, dtype='int64'), self.aspect_source

    def str_to_indexes(self, s):
        """
        Convert a string to character indexes based on character dictionary.
        
        Args:
            s (str): String to be converted to indexes
        Returns:
            str2idx (np.ndarray): Indexes of characters in s
        """
        s = s.lower()
        max_length = min(len(s), self.length)
        str2idx = np.zeros(self.length, dtype='int64')
        for i in range(1, max_length + 1):
            c = s[-i]
            if c in self.dict:
                str2idx[i - 1] = self.dict[c]
        return str2idx

In [75]:
training_data = Data(    
                         data_source=input_text,
                         aspect_source=aspect_encoded,
                         alphabet=alphabet,
                         input_size=1014,
                         num_of_classes=27)
training_data.load_data()
training_inputs, training_labels = training_data.get_all_data()

Data loaded 


In [77]:
validation_data = Data(    data_source=validation_text,
                           aspect_source=h_aspect_encoding,
                           alphabet=alphabet,
                           input_size=1014,
                           num_of_classes=27)
validation_data.load_data()
validation_inputs, validation_labels = validation_data.get_all_data()

Data loaded 


In [78]:
len(training_inputs[0])

1014

In [91]:
from keras.layers import Convolution1D
from keras.layers import AlphaDropout
from keras.callbacks import TensorBoard
from keras.layers import GlobalMaxPooling1D
from keras.layers import Embedding

class CharCNNKim(object):
   
    def __init__(self,optimizer='adam', loss='categorical_crossentropy'):
        """
        Initialization for the Character Level CNN model.
        Args:
            input_size (int): Size of input features
            alphabet_size (int): Size of alphabets to create embeddings for
            embedding_size (int): Size of embeddings
            conv_layers (list[list[int]]): List of Convolution layers for model
            fully_connected_layers (list[list[int]]): List of Fully Connected layers for model
            num_of_classes (int): Number of classes in data
            dropout_p (float): Dropout Probability
            optimizer (str): Training optimizer
            loss (str): Loss function
        """
        self.input_size = 1014
        self.alphabet_size = 69
        self.embedding_size = 128
        self.conv_layers =[
                            [
                                256,
                                10
                              ],
                              [
                                256,
                                7
                              ],
                              [
                                256,
                                5
                              ],
                              [
                                256,
                                3
                              ]
                            ]
        self.fully_connected_layers = [1024,1024]
        self.num_of_classes =27
        self.dropout_p = 0.5
        self.optimizer = optimizer
        self.loss = loss
        self._build_model()  # builds self.model variable

    def _build_model(self):
        """
        Build and compile the Character Level CNN model
        Returns: None
        """
        # Input layer
        inputs = Input(shape=(self.input_size,), name='sent_input', dtype='int64')
        # Embedding layers
        x = Embedding(self.alphabet_size + 1, self.embedding_size, input_length=self.input_size)(inputs)
        # Convolution layers
        convolution_output = []
        for num_filters, filter_width in self.conv_layers:
            conv = Convolution1D(filters=num_filters,
                                 kernel_size=filter_width,
                                 activation='tanh',
                                 name='Conv1D_{}_{}'.format(num_filters, filter_width))(x)
            pool = GlobalMaxPooling1D(name='MaxPoolingOverTime_{}_{}'.format(num_filters, filter_width))(conv)
            convolution_output.append(pool)
        x = Concatenate()(convolution_output)
        # Fully connected layers
        for fl in self.fully_connected_layers:
            x = Dense(fl, activation='selu', kernel_initializer='lecun_normal')(x)
            x = AlphaDropout(self.dropout_p)(x)
        # Output layer
        predictions = Dense(self.num_of_classes, activation='softmax')(x)
        # Build and compile model
        model = Model(inputs=inputs, outputs=predictions)
        model.compile(optimizer=self.optimizer, loss=self.loss)
        self.model = model
        print("CharCNNKim model built: ")
        self.model.summary()


    def train(self, training_inputs, training_labels,
              validation_inputs, validation_labels,
              epochs, batch_size, checkpoint_every=100):
        """
        Training function
        Args:
            training_inputs (numpy.ndarray): Training set inputs
            training_labels (numpy.ndarray): Training set labels
            validation_inputs (numpy.ndarray): Validation set inputs
            validation_labels (numpy.ndarray): Validation set labels
            epochs (int): Number of training epochs
            batch_size (int): Batch size
            checkpoint_every (int): Interval for logging to Tensorboard
        Returns: None
        """
        tensorboard = TensorBoard(log_dir='./logs', histogram_freq=checkpoint_every, batch_size=batch_size,
                                  write_graph=False, write_grads=True, write_images=False,
                                  embeddings_freq=checkpoint_every,
                                  embeddings_layer_names=None)
        # Start training
        print("Training CharCNNKim model: ")
        self.model.fit(training_inputs, training_labels,
                       validation_data=(validation_inputs, validation_labels),
                       epochs=epochs,
                       batch_size=batch_size,
                       verbose=2,
                       callbacks=[tensorboard])

    def test(self, testing_inputs, testing_labels, batch_size):
        """
        Testing function
        Args:
            testing_inputs (numpy.ndarray): Testing set inputs
            testing_labels (numpy.ndarray): Testing set labels
            batch_size (int): Batch size
        Returns: None
        """
        # Evaluate inputs
        self.model.evaluate(testing_inputs, testing_labels, batch_size=batch_size, verbose=1)
        # self.model.predict(testing_inputs, batch_size=batch_size, verbose=1)

In [92]:
alphabet = "abcdefghijklmnopqrstuvwxyz0123456789-,;.!?:'\"/\\|_@#$%^&*~`+-=<>()[]{}"
dictionary=dict()
for idx, char in enumerate(alphabet):
            dictionary[char] = idx + 1

In [93]:
model = CharCNNKim()

CharCNNKim model built: 
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
sent_input (InputLayer)         (None, 1014)         0                                            
__________________________________________________________________________________________________
embedding_8 (Embedding)         (None, 1014, 128)    8960        sent_input[0][0]                 
__________________________________________________________________________________________________
Conv1D_256_10 (Conv1D)          (None, 1005, 256)    327936      embedding_8[0][0]                
__________________________________________________________________________________________________
Conv1D_256_7 (Conv1D)           (None, 1008, 256)    229632      embedding_8[0][0]                
____________________________________________________________________________________

In [95]:
model.train(training_inputs=training_inputs,
                training_labels=training_labels,
                validation_inputs=validation_inputs,
                validation_labels=validation_labels,
                epochs=30,
                batch_size=128,
                checkpoint_every=100)


Training CharCNNKim model: 
Train on 1343 samples, validate on 93 samples
Epoch 1/30


KeyboardInterrupt: 

In [ ]:
model.test(testing_inputs=validation_inputs, 
           testing_labels=validation_labels, 
           batch_size=128)